In [1]:
# created with love in cooperation with ASPIS
from tensorflow import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Input, Conv1D, LSTM, MaxPooling1D, Flatten, concatenate, TimeDistributed, Bidirectional, ConvLSTM2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error
import os
import numpy as np

# Load data

In [2]:
f_name = "full-multinn/shift-15-windows-45"
feature_name = "Kp_index"

In [3]:
X_train = np.load(f'../data-preparation/{f_name}/X_train.npy',allow_pickle=True).item()
y_train = np.load(f'../data-preparation/{f_name}/y_train.npy') #data/lightning/ASPIS/data-preparation/archive/13500-13504.parquet.gzip/shift-15-windows-15/X_train.npy

In [4]:
Phi60_Sig1 = X_train["Phi60_Sig1"]
feature = X_train[feature_name]

# Model

In [5]:
input_feature = Input(shape=(feature.shape[1],1))
input_Phi60 = Input(shape=(Phi60_Sig1.shape[1],1))

a = Bidirectional(LSTM(64,dropout=0.1,recurrent_dropout=0.1))(input_Phi60)
a = Dense(32, activation='relu')(a)
a = Model(inputs=input_Phi60, outputs=a)

b = Bidirectional(LSTM(64,dropout=0.1,recurrent_dropout=0.1))(input_feature)
b = Dense(32, activation='relu')(b)
b = Model(inputs=input_feature, outputs=b)

x = concatenate([a.output, b.output])
x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)

output = Dense(1, activation='linear')(x)

classifier = Model(inputs=[a.input, b.input], outputs=output)
classifier.compile(loss='mse', optimizer='adam', metrics=["mae"])
print(classifier.summary())

2023-01-20 08:08:09.406882: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6233 MB memory:  -> device: 0, name: Quadro P4000, pci bus id: 0000:8b:00.0, compute capability: 6.1


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 45, 1)]      0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 45, 1)]      0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 128)          33792       ['input_2[0][0]']                
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 128)         33792       ['input_1[0][0]']                
 )                                                                                          

In [6]:
# callbacks
os.makedirs(f_name, exist_ok=True)
saved_model = "{}/epoch-{epoch:02d}-mae-{val_mae:.4f}.hdf5".format(feature_name)
checkpoint = ModelCheckpoint(f_name+"/multiNN"+saved_model, monitor='val_mae', verbose=1, save_freq="epoch")
callbacks_list = [checkpoint]

In [7]:
history = classifier.fit(
    x=[Phi60_Sig1, feature], 
    y=y_train, 
    validation_split=0.1, 
    epochs=5, 
    verbose=1, 
    callbacks=callbacks_list, 
    shuffle = True
)

Epoch 1/5
5724/5724 [==============================] - ETA: 0s - loss: 0.0149 - mae: 0.0673
Epoch 1: saving model to full-multinn/shift-15-windows-45/multiNN/epoch-01-mae-0.0563.hdf5
5724/5724 [==============================] - 3095s 539ms/step - loss: 0.0149 - mae: 0.0673 - val_loss: 0.0070 - val_mae: 0.0563
Epoch 2/5
5724/5724 [==============================] - ETA: 0s - loss: 0.0147 - mae: 0.0664
Epoch 2: saving model to full-multinn/shift-15-windows-45/multiNN/epoch-02-mae-0.0556.hdf5
5724/5724 [==============================] - 3073s 537ms/step - loss: 0.0147 - mae: 0.0664 - val_loss: 0.0070 - val_mae: 0.0556
Epoch 3/5
5724/5724 [==============================] - ETA: 0s - loss: 0.0147 - mae: 0.0661
Epoch 3: saving model to full-multinn/shift-15-windows-45/multiNN/epoch-03-mae-0.0542.hdf5
5724/5724 [==============================] - 3077s 538ms/step - loss: 0.0147 - mae: 0.0661 - val_loss: 0.0069 - val_mae: 0.0542
Epoch 4/5
5724/5724 [==============================] - ETA: 0s - lo